In [1]:
import warnings
warnings.filterwarnings('ignore')

from pymongo import MongoClient
import pprint

import copy
import pandas as pd

import requests
import numpy as np

from bs4 import BeautifulSoup

# Summary

I will now scrape data from astro-seek.com.

I will store the birthplace and birthtime in a list each.

I will add these lists to a pandas dataframe (viz. a copy of the dataframe created by newest_celeb_list.csv), and I will export it as a csv.

To make things easier, I will go one occupation at a time.

## Choosing an Occupation

I will now choose a couple of occupations.

In [2]:
df = pd.read_csv('newest_celeb_list.csv')

In [3]:
df.head()

,occupation,name_and_date,url_list
0,r_astrologers,"Stephen Arroyo (*Oct 6, 1946)",https://birthcharts.astro-seek.com/birth-chart...
1,r_astrologers,"Catherine Aubier (*Nov 8, 1941)",https://birthcharts.astro-seek.com/birth-chart...
2,r_astrologers,"Nick Campion (*Mar 4, 1953)",https://birthcharts.astro-seek.com/birth-chart...
3,r_astrologers,"Charubel (*Nov 9, 1826)",https://birthcharts.astro-seek.com/birth-chart...
4,r_astrologers,"Wim Van Dam (*Jul 26, 1950)",https://birthcharts.astro-seek.com/birth-chart...


In [6]:
df.occupation.unique()

array(['r_astrologers', 'r_chemists', 'r_cooks', 'r_diplomats',
       'r_doctors', 'r_entrepreneurs', 'r_fashion-models',
       'r_journalists-publicists', 'r_lawyers', 'r_mathematicians',
       'r_philosophers', 'r_physicists', 'r_pilots-aviators',
       'r_politicians', 'r_porn-stars', 'r_presenters', 'r_presidents',
       'r_priests-theologians', 'r_producers', 'r_psychologists',
       'r_rulers', 'r_scientists-inventors', 'r_soldiers',
       'r_stunt-performers', 'r_teachers', 'r_the-popes', 'r_translators',
       'r_voice-actors', 'r_athletes', 'r_baseball-players',
       'r_basketball-players', 'r_figure-skaters', 'r_football-players',
       'r_golfers', 'r_gymnasts', 'r_ice-hockey-players',
       'r_martial-artists', 'r_racers', 'r_runners-sprinters', 'r_skiers',
       'r_swimmers', 'r_tennis-players', 'r_actors', 'r_architects',
       'r_artists', 'r_cameramans', 'r_composers', 'r_conductors',
       'r_dancers', 'r_designers', 'r_directors', 'r_musicians',
       

## I'm Gonna Choose a Few Occupations

I will choose 15 to be exact.

My list:
-chemists
-cooks
-doctors
-entrepreneurs
-fashion models
-journalists/publicists
-lawyers
-mathematicians
-pilots/aviators
-philosophers
-physicists
-politicians
-porn stars
-presenters
-presidents
-priests/theologians
-producers
-rulers
-scientists/inventors
-soldiers
-baseball players
-basketball players
-martial artists
-racers
-tennis players
-actors (shorten this list though)
-architects
-artists
-composers
-dancers
-designers
-directors (shorten this list though)
-musicians
-painters
-photographers
-poets (cut this list in half)
-script writers (shorten this list though)
-sculptors
-singers
-writers (cut this list in half)
-activists
-founders

Ok, I need to shorten this list. First, I'll organize it into categories:


SCIENCE
-scientists/inventors
-chemists
-cooks
-doctors
-physicists


BUSINESS
-entrepreneurs
-designers
-architects


MODELING
-fashion models
-porn stars



WRITER
-journalists/publicists
-poets (cut this list in half)
-writers (cut this list in half)



ARTS
-artists
-painters
-photographers
-sculptors





LEADER
-rulers
-presidents
-priests/theologians
-politicians
-founders



ACADEMIC
-mathematicians
-philosophers



ATHLETE
-baseball players
-basketball players
-martial artists
-racers
-tennis players



FINE_ARTS/CINEMA
-actors (shorten this list though)
-producers
-script writers (shorten this list though)
-dancers
-directors (shorten this list though)




MUSIC
-singers
-musicians
-composers



OTHER
-lawyers
-pilots/aviators
-soldiers
-activists
-presenters




...Ok, well that's 42 occupations. That's fine I guess

In [8]:
df.occupation.unique()

array(['r_astrologers', 'r_chemists', 'r_cooks', 'r_diplomats',
       'r_doctors', 'r_entrepreneurs', 'r_fashion-models',
       'r_journalists-publicists', 'r_lawyers', 'r_mathematicians',
       'r_philosophers', 'r_physicists', 'r_pilots-aviators',
       'r_politicians', 'r_porn-stars', 'r_presenters', 'r_presidents',
       'r_priests-theologians', 'r_producers', 'r_psychologists',
       'r_rulers', 'r_scientists-inventors', 'r_soldiers',
       'r_stunt-performers', 'r_teachers', 'r_the-popes', 'r_translators',
       'r_voice-actors', 'r_athletes', 'r_baseball-players',
       'r_basketball-players', 'r_figure-skaters', 'r_football-players',
       'r_golfers', 'r_gymnasts', 'r_ice-hockey-players',
       'r_martial-artists', 'r_racers', 'r_runners-sprinters', 'r_skiers',
       'r_swimmers', 'r_tennis-players', 'r_actors', 'r_architects',
       'r_artists', 'r_cameramans', 'r_composers', 'r_conductors',
       'r_dancers', 'r_designers', 'r_directors', 'r_musicians',
       

In [14]:
occupations_list_shortened = [
'r_scientists-inventors',
'r_chemists',
'r_cooks',
'r_doctors',
'r_physicists',
'r_entrepreneurs',
'r_designers',
'r_architects',
'r_fashion-models',
'r_porn-stars',
'r_journalists-publicists',
'r_poets',
'r_writers',
'r_artists',
'r_painters',
'r_photographers',
'r_sculptors',
'r_rulers',
'r_presidents',
'r_priests-theologians',
'r_politicians',
'r_founders',
'r_mathematicians',
'r_philosophers',
'r_basketball-players',
'r_baseball-players',
'r_martial-artists',
'r_racers',
'r_tennis-players',
'r_actors',
'r_producers',
'r_script-writers',
'r_dancers',
'r_directors-ceos',
'r_singers',
'r_musicians',
'r_composers',
'r_lawyers',
'r_pilots-aviators',
'r_soldiers',
'r_activists',
'r_presenters']

In [15]:
print(occupations_list_shortened)

['r_scientists-inventors', 'r_chemists', 'r_cooks', 'r_doctors', 'r_physicists', 'r_entrepreneurs', 'r_designers', 'r_architects', 'r_fashion-models', 'r_porn-stars', 'r_journalists-publicists', 'r_poets', 'r_writers', 'r_artists', 'r_painters', 'r_photographers', 'r_sculptors', 'r_rulers', 'r_presidents', 'r_priests-theologians', 'r_politicians', 'r_founders', 'r_mathematicians', 'r_philosophers', 'r_basketball-players', 'r_baseball-players', 'r_martial-artists', 'r_racers', 'r_tennis-players', 'r_actors', 'r_producers', 'r_script-writers', 'r_dancers', 'r_directors-ceos', 'r_singers', 'r_musicians', 'r_composers', 'r_lawyers', 'r_pilots-aviators', 'r_soldiers', 'r_activists', 'r_presenters']


Ok, now we make a function which accepts an occupation and creates an appropriate dataframe.

My idea is to mark the index of the first instance and of the last instance for each occupation.

Afterwards, we just scrape the urls from those rows.

In [18]:
first_and_last_row_dict = {}

for o in occupations_list_shortened:
    print(o)
    temp_idx = np.where(df['occupation'] == o)
    #print(temp_idx)
    temp_val_i = temp_idx[0][0].item()
    print(temp_val_i)
    temp_val_ii = temp_idx[0][-1].item()
    print(temp_val_ii)
    first_and_last_row_dict[o] = [temp_val_i, temp_val_ii]
    
print(first_and_last_row_dict)

r_scientists-inventors
11140
11371
r_chemists
957
1128
r_cooks
1129
1340
r_doctors
1434
1587
r_physicists
4802
5217
r_entrepreneurs
1588
2105
r_designers
70129
70393
r_architects
65332
65691
r_fashion-models
2106
3528
r_porn-stars
8347
8637
r_journalists-publicists
3529
4295
r_poets
86097
86814
r_writers
98771
103668
r_artists
65692
66157
r_painters
85235
85951
r_photographers
85952
86096
r_sculptors
92268
92406
r_rulers
11030
11139
r_presidents
9150
9357
r_priests-theologians
9358
9540
r_politicians
5386
8346
r_founders
104031
104230
r_mathematicians
4468
4588
r_philosophers
4589
4801
r_basketball-players
13490
14169
r_baseball-players
13043
13489
r_martial-artists
15848
16279
r_racers
16280
16568
r_tennis-players
16963
17347
r_actors
17348
65331
r_producers
9541
10900
r_script-writers
86815
92267
r_dancers
69432
70128
r_directors-ceos
103986
104030
r_singers
92407
98770
r_musicians
83079
85157
r_composers
67077
69326
r_lawyers
4296
4467
r_pilots-aviators
5218
5385
r_soldiers
11372
11

In [26]:
# check...
df.iloc[11717:11719]

,occupation,name_and_date,url_list
11717,r_soldiers,"Josef Mašín (*Aug 26, 1896)",https://birthcharts.astro-seek.com/birth-chart...
11718,r_stunt-performers,"Jackie Chan (*Apr 7, 1954)",https://birthcharts.astro-seek.com/birth-chart...


In [27]:
client = MongoClient('localhost', 27017)
db = client.metroid
pages = db.pages

In [83]:
first_and_last_row_dict['r_artists'][0]

65692

In [84]:
first_and_last_row_dict[occupations_list_shortened[0]][0]

11140

In [147]:
temporary_df_dict = {}

In [ ]:
occupations_list_shortened_var = [occupations_list_shortened[_] for _ in range(29,42)]
print(occupations_list_shortened_var)


# For loop over each occupation.
for o in occupations_list_shortened_var:
    o_info_list = []
    o_r_dict = {}
    
    o_name_list = []
    o_occupation_list = []
    
    lower = first_and_last_row_dict[o][0]
    higher = first_and_last_row_dict[o][1] + 1
    
    if higher - lower > 1000:
        higher = lower + 1000
    
    for i in range(lower, higher):
    #for i in range(lower, lower + 3):
        proto_celeb_name = df.iloc[i]['name_and_date']
        hmm = proto_celeb_name.index('(')
        almost_celeb_name = proto_celeb_name[:hmm - 1]
        celeb_name = almost_celeb_name.replace(' ', '_')
        
        o_name_list.append(celeb_name)
        o_occupation_list.append(o)
        
        r = 'r_' + celeb_name
        current_celeb_url = df.iloc[i]['url_list']
        r_val = requests.get(current_celeb_url)
        #print(r + ' is returning ' + str(r_val))
        o_r_dict[r] = r_val
        pages.insert_one({'html': r_val.content})

        soup = BeautifulSoup(r_val.content, 'html')
        
        div = soup.find('div', {'style': 'float: left; width: 310px; overflow: hidden; background: #;'})
        try:
            t = div.text.strip()
            t = t.replace('\n', '')
            #print(t)
            o_info_list.append(t)
        except:
            o_info_list.append('no_info_found')
    
    #temporary_df_dict[o + str('_df')] = pd.DataFrame([o_info_list]).transpose()
    
    #zippedLList =  list(zip(occupation_llllist, name_and_date_llllist, name_url_llllist))
    #da_bestest_df = pd.DataFrame(zippedLList, columns = ['occupation', 'name_and_date', 'url_list'])
    
    zipped_list = list(zip(o_name_list, o_info_list, o_occupation_list))
    temporary_df_dict[o + str('_df')] = pd.DataFrame(zipped_list, columns = ['name', 'info', 'occupation'])


['r_actors', 'r_producers', 'r_script-writers', 'r_dancers', 'r_directors-ceos', 'r_singers', 'r_musicians', 'r_composers', 'r_lawyers', 'r_pilots-aviators', 'r_soldiers', 'r_activists', 'r_presenters']


In [146]:
#temporary_df_dict.keys()
#temporary_df_dict['r_fashion-models_df']
#temporary_df_dict
#temporary_df_dict['r_tennis-players_df']

{}

In [143]:
temporary_df_dict['r_tennis-players_df'].to_csv('hi_its_tennis-players_df.csv', index=False)